In [1]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import ensemble
from math import sqrt
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import BaggingClassifier

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
#----------------------------------------------------------------------------------------------------資料前處理
train_x = train[['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
Std = StandardScaler()
train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']] = Std.fit_transform(train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']]) #標準化
train_x[['HasCrCard','IsActiveMember']] = train_x[['HasCrCard','IsActiveMember']].astype(str)
train_x_dummies = pd.get_dummies(train_x)
print(train_x_dummies)
train_y = train['Exited']
#------------------------------------------------------------------------------------------------------------------------------
test_x = test[['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
test_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']] = Std.fit_transform(test_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']]) #標準化
test_x[['HasCrCard','IsActiveMember']] = test_x[['HasCrCard','IsActiveMember']].astype(str)
test_x_dummies = pd.get_dummies(test_x)
dummy = test_x_dummies.columns.tolist()
#建立 nn 模型 0.7411 0.7273------rank2-------------------------------------------------------------------------------------------------------
alphas = [0.0001,0.01,1]
acti = ['identity', 'logistic', 'tanh', 'relu']
nn = MLPClassifier(alpha = alphas[2], random_state = 10, activation = acti[3],max_iter=1000)
#建立 random forest 模型 0.7437 0.7284------rank1-------------------------------------------------------------------------------------------
max_sample = [0.6,0.7,0.8]
max_feature = [round(sqrt(len(dummy))-1),round(sqrt(len(dummy))),round(sqrt(len(dummy))+1)]
forest = ensemble.RandomForestClassifier(criterion="entropy",random_state=10,max_samples=max_sample[0],max_features=max_feature[2])
#建立 decision tree 模型 0.5757 0.5669--------rank5------------------------------------------------------------------------------------------
dtree = tree.DecisionTreeClassifier(random_state=10)
#建立 logistic regression 模型 0.6118 0.5892----rank4--------------------------------------------------------------------------------------
c = [0.01,1,100,10000,1000000]
lr = LogisticRegression(C=c[2], penalty = "l2",random_state = 10)
#建立 svm 模型 0.7373 0.7214----rank3------------------------------------------------------------------------------------------------------------------
c = [0.1,1,10]
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
svm = svm.SVC(C=c[1], random_state=10, kernel=kernels[1],probability=True)
#建立 bag 模型----------------------------------------------------------------------------------------------------------------------------
bag = BaggingClassifier(n_estimators=80,max_samples=0.71 ,max_features=0.5,random_state=10)
model0 = bag.fit(train_x_dummies,train_y)
pred_y = model0.predict(test_x_dummies)
final = {"RowNumber":test["RowNumber"],'Exited':pred_y}
final_df = pd.DataFrame(final)
final_df.to_csv('bag.csv')
#建立 voting classfier
voting = ["hard","soft"]
weight1 = [[1,1,1,1,1],[2,1,2,1,1]]
#建立分類器-----------------------------
i = 0
while i < len(voting):
    j = 0
    while j < len(weight1):
        #---------------------------------------------------------------------------------------------------------------------------------------
        vc0 = ensemble.VotingClassifier(estimators=[('nn', nn), ('forest', forest), ('bag', bag), ('lr', lr), ('svm', svm)], voting=voting[i],weights=weight1[j])
        model0 = vc0.fit(train_x_dummies, train_y)
        # 預測
        test_y_predicted = model0.predict(test_x_dummies)
        predict = test_y_predicted.tolist()
        test['Exited'] = predict
        # ----------------------------------------------------------------------------------------------------------------------
        answer = {"RowNumber": test["RowNumber"], "Exited": test["Exited"]}
        answer_df = pd.DataFrame(answer)
        print(answer_df)
        answer_df.to_csv("vc" +  str(i) + str(j) + ".csv")
        j = j + 1
    i = i + 1
#-------------------------------------------------------------------------
acc = [0.8825,0.88,0.885,0.8825]
preci = [0.8333,0.814,0.8372,0.8182]
fscore = [0.5983,0.5932,0.6102,0.6050]
final = []
hw6 = []
i = 0
while i < len(acc):
    final.append(round(((acc[i] + preci[i] + fscore[i])/3),4))
    hw6.append(round((0.3 * acc[i] + 0.3 * preci[i] + 0.4 * fscore[i]),4))
    i = i + 1
print(final)
print(hw6)
answer = {"Accuracy": acc, "Precision": preci, "F-score": fscore, "Final": final, "HW6-Grade": hw6}
answer_df = pd.DataFrame(answer)
print(answer_df)
answer_df.to_csv("voting-result.csv")

<ipython-input-1-e8220bcfe55d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']] = Std.fit_transform(train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']]) #標準化
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3065: Set

      CreditScore       Age    Tenure   Balance  NumOfProducts  \
0        1.509906 -0.663721  0.339933  0.428978       0.799546   
1        0.866637  1.465241 -0.445812 -1.214512      -0.917139   
2        0.697356 -1.595141  0.339933  0.615161      -0.917139   
3       -0.656894 -0.397600 -0.445812  0.117359      -0.917139   
4        2.068534 -0.131480  0.339933  0.630562      -0.917139   
...           ...       ...       ...       ...            ...   
7995    -1.063169 -1.062901  1.125677  0.652575       0.799546   
7996    -0.487612  0.267700  1.911421  1.440568      -0.917139   
7997     0.815853  0.001580  0.339933  1.025801      -0.917139   
7998     1.425266 -0.663721 -0.445812  0.577801      -0.917139   
7999    -0.521469 -0.131480  0.339933 -1.214512      -0.917139   

      EstimatedSalary  Geography_France  Geography_Germany  Geography_Spain  \
0            0.304927                 0                  0                1   
1            0.529787                 1          